In [6]:
#import libraries
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Conv1D, SimpleRNN, Bidirectional, MaxPooling1D, GlobalMaxPool1D, LSTM, GRU
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Dropout
from keras.layers import Dense, LSTM, Embedding
from tensorflow import keras
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence
from sklearn.utils.class_weight import compute_class_weight

In [27]:
# Load the JSON file into a pandas DataFrame
data = pd.read_csv('news_category.csv')
data.head(5)

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",9/23/2022
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,9/23/2022
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,9/23/2022
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,9/23/2022
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,9/22/2022


In [28]:
# #dropping unnecessary columns
new_df = data.drop(columns=['authors','link','date'])
new_df.head()

,headline,category,short_description
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha..."
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to..."
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...


In [29]:
# create final dataframe of combined headline and short_description
final_df = new_df.copy()
final_df['length_of_news'] = final_df['headline'] + final_df['short_description']
final_df.drop(['headline','short_description'], inplace=True, axis=1)
final_df['len_news'] = final_df['length_of_news'].apply(lambda x: len(str(x)))
final_df.head()


,category,length_of_news,len_news
0,U.S. NEWS,Over 4 Million Americans Roll Up Sleeves For O...,230
1,U.S. NEWS,"American Airlines Flyer Charged, Banned For Li...",248
2,COMEDY,23 Of The Funniest Tweets About Cats And Dogs ...,133
3,PARENTING,The Funniest Tweets From Parents This Week (Se...,215
4,U.S. NEWS,Woman Who Called Cops On Black Bird-Watcher Lo...,233


In [30]:
# final_df.drop(columns=['category'])
final_df.head()


,category,length_of_news,len_news
0,U.S. NEWS,Over 4 Million Americans Roll Up Sleeves For O...,230
1,U.S. NEWS,"American Airlines Flyer Charged, Banned For Li...",248
2,COMEDY,23 Of The Funniest Tweets About Cats And Dogs ...,133
3,PARENTING,The Funniest Tweets From Parents This Week (Se...,215
4,U.S. NEWS,Woman Who Called Cops On Black Bird-Watcher Lo...,233


In [31]:
# Preprocess the data
def preprocess_text(text):
    # Remove special characters and digits
    text = re.sub('[^a-zA-Z]', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Split into words
    words = text.split()
    # Join the words
    preprocessed_text = ' '.join(words)
    return preprocessed_text

final_df['length_of_news'] = final_df['length_of_news'].apply(lambda x: preprocess_text(str(x)))


In [32]:
final_df['category_merged']=final_df['category'].replace({"HEALTHY LIVING": "WELLNESS",
"QUEER VOICES": "GROUPS VOICES",
"BUSINESS": "BUSINESS & FINANCES",
"PARENTS": "PARENTING",
"BLACK VOICES": "GROUPS VOICES",
"THE WORLDPOST": "WORLD NEWS",
"STYLE": "STYLE & BEAUTY",
"GREEN": "ENVIRONMENT",
"TASTE": "FOOD & DRINK",
"WORLDPOST": "WORLD NEWS",
"SCIENCE": "SCIENCE & TECH",
"TECH": "SCIENCE & TECH",
"MONEY": "BUSINESS & FINANCES",
"ARTS": "ARTS & CULTURE",
"COLLEGE": "EDUCATION",
"LATINO VOICES": "GROUPS VOICES",
"CULTURE & ARTS": "ARTS & CULTURE",
"FIFTY": "MISCELLANEOUS",
"GOOD NEWS": "MISCELLANEOUS"})

In [33]:
# one hot encoding using keras tokenizer and pad sequencing
X = final_df['length_of_news']
encoder = LabelEncoder()
y = encoder.fit_transform(final_df['category_merged'])
print("shape of input data: ", X.shape)
print("shape of target variable: ", y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
tokenizer = Tokenizer(num_words=10000, oov_token='<00V>') 
tokenizer.fit_on_texts(X_train) # build the word index

# train_seq = tokenizer.texts_to_sequences(train_data)
# test_seq = tokenizer.texts_to_sequences(test_data)

print(type(X))

shape of input data:  (209527,)
shape of target variable:  (209527,)
<class 'pandas.core.series.Series'>


In [38]:
print(len(y))

209527


In [44]:
# padding X_train text input data
train_seq = tokenizer.texts_to_sequences(X_train) # converts strinfs into integer lists
train_padseq = pad_sequences(train_seq, maxlen=150) # pads the integer lists to 2D integer tensor 

# padding X_test text input data
test_seq = tokenizer.texts_to_sequences(X_test)
test_padseq = pad_sequences(test_seq, maxlen=150)
print(train_padseq.shape)



(167621, 150)


In [45]:
word_index = tokenizer.word_index
max_words = 50000  # total number of words to consider in embedding layer
total_words = len(word_index)
maxlen = 150 # max length of sequence 
num_classes = len(final_df['category_merged'].unique())
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

print("Length of word index:", total_words)

Length of word index: 139620


In [46]:
print(final_df['category_merged'].unique())

['U.S. NEWS' 'COMEDY' 'PARENTING' 'WORLD NEWS' 'ARTS & CULTURE'
 'SCIENCE & TECH' 'SPORTS' 'ENTERTAINMENT' 'POLITICS' 'WEIRD NEWS'
 'ENVIRONMENT' 'EDUCATION' 'CRIME' 'WELLNESS' 'BUSINESS & FINANCES'
 'STYLE & BEAUTY' 'FOOD & DRINK' 'MEDIA' 'GROUPS VOICES' 'HOME & LIVING'
 'WOMEN' 'TRAVEL' 'RELIGION' 'IMPACT' 'WEDDINGS' 'MISCELLANEOUS' 'DIVORCE'
 nan]


In [47]:
print(np.shape(X_train))
print(y_train.shape)
from tensorflow.keras.layers import Input
# input_layer = Input(shape=(maxlen,))
input_layer = Input(shape=(20,))

(167621,)
(167621, 28, 28)


In [48]:
from tensorflow.keras.layers import Reshape
# basline model using embedding layers and simpleRNN
model = Sequential()
model.add(Embedding(max_words, 70, input_length=maxlen))
model.add(Bidirectional(SimpleRNN(64, dropout=0.1, recurrent_dropout=0.20, activation='tanh', return_sequences=True)))
model.add(Bidirectional(SimpleRNN(64, dropout=0.1, recurrent_dropout=0.30, activation='tanh', return_sequences=True)))
model.add(SimpleRNN(32, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(28, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 70)           3500000   
                                                                 
 bidirectional (Bidirectiona  (None, 150, 128)         17280     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 150, 128)         24704     
 nal)                                                            
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 32)                5152      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 28)                9

In [45]:
#Train the model
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )
# SETUP A EARLY STOPPING CALL and model check point API
earlystopping = keras.callbacks.EarlyStopping(monitor='accuracy',
                                              patience=5,
                                              verbose=1,
                                              mode='min'
                                              )
checkpointer = ModelCheckpoint(filepath='bestvalue',monitor='val_loss', verbose=0, save_best_only=True)
callback_list = [checkpointer, earlystopping]
# fit model to the data
history = model.fit(train_padseq, y_train, 
                     batch_size=128, 
                     epochs=10, 
                     validation_split=0.2,
                     shuffle=True
                    )

# evalute the model
test_loss, test_acc = model.evaluate(test_padseq, y_test, verbose=0)
print("test loss and accuracy:", test_loss, test_acc)




Epoch 1/10
1048/1048 [==============================] - 590s 559ms/step - loss: 2.5400 - accuracy: 0.3423 - val_loss: 2.3420 - val_accuracy: 0.3769
Epoch 2/10
1048/1048 [==============================] - 722s 689ms/step - loss: 2.3054 - accuracy: 0.3949 - val_loss: 2.1756 - val_accuracy: 0.4263
Epoch 3/10
1048/1048 [==============================] - 747s 713ms/step - loss: 2.1824 - accuracy: 0.4278 - val_loss: 2.1309 - val_accuracy: 0.4291
Epoch 4/10
1048/1048 [==============================] - 713s 680ms/step - loss: 2.1053 - accuracy: 0.4446 - val_loss: 2.1203 - val_accuracy: 0.4437
Epoch 5/10
1048/1048 [==============================] - 846s 807ms/step - loss: 2.0549 - accuracy: 0.4593 - val_loss: 2.0312 - val_accuracy: 0.4667
Epoch 6/10
1048/1048 [==============================] - 799s 762ms/step - loss: 2.0163 - accuracy: 0.4713 - val_loss: 2.0028 - val_accuracy: 0.4723
Epoch 7/10
1048/1048 [==============================] - 763s 728ms/step - loss: 1.9943 - accuracy: 0.4799 - val_

In [49]:
import pickle

In [50]:
filename = 'model.sav'
pickle.dump(model, open(filename, 'wb'))

In [51]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

AttributeError: 'Sequential' object has no attribute 'score'

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import normalize
import pandas as pd
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [7]:
model = tf.keras.models.load_model('model_73_percent.h5')

NameError: name 'tf' is not defined

In [4]:
data='Philadelphia Phillies first baseman and slugger Rhys Hoskins suffered a torn anterior cruciate ligament (ACL) in his left knee during a spring training game against the Detroit Tigers on March 23, 2023. He underwent surgery to repair the injury on March 28, 2023. The initial expectation was that Hoskins would miss the entire 2023 season, but he himself raised the possibility of returning in time for the postseason. Hoskins is scheduled to become a free agent after the season. The Phillies will have to find a replacement for Hoskins in their lineup, and some options include rotating between Darick Hall and Alec Bohm'

In [9]:
data = pd.Series(data)
data

0    Philadelphia Phillies first baseman and slugge...
dtype: object

In [10]:
tokenizer = Tokenizer(num_words=10000, oov_token='<00V>') 
tokenizer.fit_on_texts(data)
train_seq = tokenizer.texts_to_sequences(data)
train_padseq = pad_sequences(train_seq, maxlen=130)
train_padseq

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 15,  9, 16, 17,  5,
        18, 19,  3, 20,  4, 21, 22, 23, 24, 25,  6, 26, 27, 28, 29,  4,
        30, 31, 32, 33,  2, 34, 35, 10, 11, 36,  7, 12, 37, 38,  8, 39,
         2, 40, 10, 11, 41,  7,  2, 42, 43, 44, 45,  3, 46, 47,  2, 48,
         7, 13, 49, 12, 50, 51,  2, 52, 53, 54,  6, 55, 14,  2, 56,  3,
        57, 58,  8, 59,  4, 60, 61, 62,  2, 13,  2,  9, 63, 64,  8, 65,
         4, 66, 14,  3,  6, 67, 68,  5, 69, 70, 71, 72, 73, 74, 75,  5,
        76, 77]])

In [22]:
# category=({"HEALTHY LIVING": "WELLNESS",
# "QUEER VOICES": "GROUPS VOICES",
# "BUSINESS": "BUSINESS & FINANCES",
# "PARENTS": "PARENTING",
# "BLACK VOICES": "GROUPS VOICES",
# "THE WORLDPOST": "WORLD NEWS",
# "STYLE": "STYLE & BEAUTY",
# "GREEN": "ENVIRONMENT",
# "TASTE": "FOOD & DRINK",
# "WORLDPOST": "WORLD NEWS",
# "SCIENCE": "SCIENCE & TECH",
# "TECH": "SCIENCE & TECH",
# "MONEY": "BUSINESS & FINANCES",
# "ARTS": "ARTS & CULTURE",
# "COLLEGE": "EDUCATION",
# "LATINO VOICES": "GROUPS VOICES",
# "CULTURE & ARTS": "ARTS & CULTURE",
# "FIFTY": "MISCELLANEOUS",
# "GOOD NEWS": "MISCELLANEOUS"})


In [13]:
prediction=model.predict(train_padseq)

1/1 [==============================] - 0s 121ms/step


In [20]:
print(len(prediction[0]))

27


In [17]:
import numpy as np
encoded_argmax  = np.argmax(prediction, axis=1)
text = tokenizer.sequences_to_texts([encoded_argmax])

In [21]:
print(encoded_argmax)

[14]


In [24]:
print(prediction[0])

[0.35952905 0.5498701  0.4935158  0.01783587 0.33507463 0.5743584
 0.16493218 0.07801519 0.10505788 0.4109367  0.21230389 0.7148832
 0.07696788 0.6360627  0.9371359  0.35498768 0.51193184 0.14270647
 0.05731709 0.44376197 0.5340759  0.00820367 0.43864402 0.01987911
 0.9084652  0.911222   0.07319453]
